# 확률 네트워크기반 친환경 자동차 충전소(GVCS) 로케이션 인텔리전스(Location Intelligence) 대시보드

## 제안 배경

**친환경 자동차 생산에만 관심이 쏠려 사용자를 고려하지 못한 설치로 인한 잉여 충전소 발생**

- 민관 협력을 통해 충전 서비스 관련 사업이 꾸준히 증가함. 하지만 전국에 설치된 공공 급속충전기 총 2,896개(2021년 기준) 중 하루 1회 이하로 사용한 충전기가 1,164개로 전체의 40.1%를 차지하고 있다. 심지어 17개는 전혀 사용하지 않은 것으로 나타남.

- 이와 같은 충전소마다 이용 편차가 큰 이유는 전기차 보급 목표에 따라 구역별로  충전소를 나눠 설치했기 때문이라는 지적도 있다.

- 한국전기차충전협동조합은 앞으로 수요가 몰리는 거점을 중심을 중심으로 충전기 설치를 해야하 한다는 의견 주장함.

- 기존의 정부 중심으로 공급자 입장에서 설치하던 충전소를 친환경 자동차 차주를 고려해 이용자 중심으로 변화 설치가 필요하다는 지적도 있다.

즉, **꾸준히 증가하는 친환경 차량의 수요를 위해 저비용 고효율의 충전소 설치를 목표 한다.** 충전소 설치 사업자의 기대 수익 최적화를 위해 사용자 입장을 고려한 체계적이며 최적의 입지 선정 결과를 제시함.

## 사용 데이터

1. 충전소 위치 및 현황 정보
2. 급속충전기 보급 현황
3. LPG충전소 현황
4. 오존 월별 대기오염도
5. 아황산가스 월별 대기오염도
6. 이산화질소 월별 대기오염도
7. 미세먼지 월별 대기오염도
8. 행정구역별 인구수
9. 항만 현황
10. 지역별 전기차 등록대 수

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import osmnx as ox
import networkx as nx
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

## 상위 요소 - 환경적 요인

- 오존
- 미세먼지
- 아황산가스
- 이산화질소
- 일산화탄소
- 주차장 정보

### 오존 월별 대기오염도 데이터 불러오기

In [2]:
ozone_file_path = "C:\\Users\\pwjdg\\바탕 화면\\Green-Car-Charging-Station-Site-Selection\\Data\\오존_월별_도시별_대기오염도.csv"
df_ozone = pd.read_csv(ozone_file_path, encoding='cp949')
df_ozone

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,0.029,0.034,0.043,0.042,0.044,0.033
1,서울특별시,서울특별시,0.022,0.025,0.037,0.036,0.041,0.038
2,부산광역시,부산광역시,0.031,0.036,0.043,0.041,0.041,0.028
3,대구광역시,대구광역시,0.025,0.032,0.043,0.040,0.045,0.031
4,인천광역시,인천광역시,0.026,0.030,0.040,0.042,0.044,0.041
...,...,...,...,...,...,...,...,...
167,경상남도,합천,0.034,0.039,0.050,0.048,0.051,0.038**
168,경상남도,도평균,0.032,0.037,0.044,0.044,0.045,0.031
169,제주특별자치도,제주특별자치도,0.044,0.046,0.051,0.051,0.045,0.029
170,제주특별자치도,서귀포,0.045,0.048,0.054,0.052,0.046,0.028


In [3]:
df_ozone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   구분(1)    172 non-null    object
 1   구분(2)    172 non-null    object
 2   2021.02  172 non-null    object
 3   2021.03  172 non-null    object
 4   2021.04  172 non-null    object
 5   2021.05  172 non-null    object
 6   2021.06  172 non-null    object
 7   2021.07  172 non-null    object
dtypes: object(8)
memory usage: 10.9+ KB


In [4]:
ozen_col =  df_ozone.iloc[:,2:-1].columns.tolist()

In [5]:
df_ozone.drop(columns=ozen_col, inplace=True)

In [6]:
def advanced_replace(df, col, str, regex):
    '''
    DataFrame 특정 열의 특수문자 제거
    :param df: 대상 DataFrame
    :param col: column
    :param str: 제거할 특수 문자.
    :param regex: 적용할 정규표현식
    :return:
    '''
    df[col] = df[col].replace(str,0)
    df[col] = df[col].replace(to_replace=regex, value=r'', regex=True)
    return df

In [7]:
# 해당 컬럼 float 타입으로 변경
advanced_replace(df_ozone, df_ozone.iloc[:,2:].columns.tolist(), '-', r'[^0-9.0-9]')
df_ozone['2021.07'] = df_ozone['2021.07'].astype(float)
df_ozone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [8]:
df_ozone.describe()

,2021.07
count,172.000000
mean,0.031924
std,0.005394
min,0.019000
25%,0.028000
50%,0.031000
75%,0.035000
max,0.059000


### 아황산가스 월별 데이터 불러오기

In [9]:
so2_file_path = 'C:\\Users\\pwjdg\\바탕 화면\\Green-Car-Charging-Station-Site-Selection\\Data\\아황산가스_월별_도시별_대기오염도.csv'
df_so2 = pd.read_csv(so2_file_path, encoding='cp949')
df_so2

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,0.003,0.003,0.003,0.003,0.003,0.003
1,서울특별시,서울특별시,0.003,0.003,0.003,0.003,0.003,0.003
2,부산광역시,부산광역시,0.004,0.003,0.003,0.002,0.003,0.003
3,대구광역시,대구광역시,0.002,0.002,0.002,0.002,0.002,0.002
4,인천광역시,인천광역시,0.004,0.003,0.003,0.003,0.003,0.003
...,...,...,...,...,...,...,...,...
167,경상남도,합천,0.003,0.003,0.003,0.003,0.003,0.003**
168,경상남도,도평균,0.003,0.003,0.003,0.003,0.003,0.003
169,제주특별자치도,제주특별자치도,0.002,0.001,0.002,0.002,0.002,0.002
170,제주특별자치도,서귀포,0.001,0.001,0.001,0.001,0.001,0.001


In [10]:
df_so2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   구분(1)    172 non-null    object
 1   구분(2)    172 non-null    object
 2   2021.02  172 non-null    object
 3   2021.03  172 non-null    object
 4   2021.04  172 non-null    object
 5   2021.05  172 non-null    object
 6   2021.06  172 non-null    object
 7   2021.07  172 non-null    object
dtypes: object(8)
memory usage: 10.9+ KB


In [11]:
so2_col = df_so2.iloc[:,2:-1].columns.tolist()

In [12]:
df_so2.drop(columns=so2_col, inplace=True)
df_so2

,구분(1),구분(2),2021.07
0,총계,소계,0.003
1,서울특별시,서울특별시,0.003
2,부산광역시,부산광역시,0.003
3,대구광역시,대구광역시,0.002
4,인천광역시,인천광역시,0.003
...,...,...,...
167,경상남도,합천,0.003**
168,경상남도,도평균,0.003
169,제주특별자치도,제주특별자치도,0.002
170,제주특별자치도,서귀포,0.001


In [13]:
advanced_replace(df_so2,'2021.07','-',r'[^0-9.0-9]')
df_so2['2021.07'] = df_so2['2021.07'].astype(float)

In [14]:
df_so2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [15]:
df_so2.describe()

,2021.07
count,172.000000
mean,0.002674
std,0.000764
min,0.001000
25%,0.002000
50%,0.003000
75%,0.003000
max,0.005000


### 이산화질소 월별 데이터 불러오기

In [16]:
no2_file_path = 'C:\\Users\\pwjdg\\바탕 화면\\Green-Car-Charging-Station-Site-Selection\\Data\\이산화질소_월별_도시별_대기오염도.csv'
df_no2 = pd.read_csv(no2_file_path, encoding='cp949')
df_no2

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,0.019,0.019,0.014,0.012,0.011,0.010
1,서울특별시,서울특별시,0.030,0.032,0.022,0.019,0.018,0.016
2,부산광역시,부산광역시,0.018,0.017,0.015,0.014,0.014,0.010
3,대구광역시,대구광역시,0.022,0.018,0.013,0.012,0.010,0.009
4,인천광역시,인천광역시,0.024,0.028,0.021,0.017,0.017,0.014
...,...,...,...,...,...,...,...,...
167,경상남도,합천,0.014,0.011,0.010,0.009,0.010,0.009**
168,경상남도,도평균,0.016,0.014,0.013,0.011,0.012,0.009
169,제주특별자치도,제주특별자치도,0.011,0.011,0.010,0.010,0.009,0.006
170,제주특별자치도,서귀포,0.007,0.007,0.006,0.006,0.005,0.004


In [17]:
no2_col = df_no2.iloc[:,2:-1].columns.tolist()

In [18]:
df_no2.drop(columns=no2_col, inplace=True)

In [19]:
advanced_replace(df_no2,'2021.07','-',r'[^0-9.0-9]')
df_no2['2021.07'] = df_no2['2021.07'].astype(float)

In [20]:
df_no2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [21]:
df_no2.describe()

,2021.07
count,172.000000
mean,0.007610
std,0.003294
min,0.003000
25%,0.005000
50%,0.007000
75%,0.009250
max,0.019000


### 일산화탄소 월별 데이터 불러오기

In [22]:
co_file_path = 'C:\\Users\\pwjdg\\바탕 화면\\Green-Car-Charging-Station-Site-Selection\\Data\\일산화탄소_월별_도시별_대기오염도.csv'
df_co = pd.read_csv(co_file_path, encoding='cp949')
df_co

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,0.5,0.5,0.4,0.4,0.4,0.3
1,서울특별시,서울특별시,0.6,0.5,0.4,0.4,0.4,0.3
2,부산광역시,부산광역시,0.4,0.4,0.3,0.3,0.3,0.2
3,대구광역시,대구광역시,0.5,0.4,0.3,0.4,0.4,0.3
4,인천광역시,인천광역시,0.6,0.6,0.4,0.4,0.4,0.4
...,...,...,...,...,...,...,...,...
167,경상남도,합천,0.4,0.3,0.3,0.3,0.3,0.2**
168,경상남도,도평균,0.5,0.4,0.4,0.4,0.4,0.3
169,제주특별자치도,제주특별자치도,0.3,0.3,0.3,0.3,0.3,0.2
170,제주특별자치도,서귀포,0.3,0.3,0.3,0.3,0.2,0.2


In [23]:
co_col = df_co.iloc[:,2:-1].columns.tolist()

In [24]:
df_co.drop(columns=co_col, inplace=True)

In [25]:
advanced_replace(df_co, '2021.07','-',r'[^0-9.0-9]')
df_co['2021.07'] = df_co['2021.07'].astype(float)

In [26]:
df_co.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [27]:
df_co.describe()

,2021.07
count,172.000000
mean,0.328488
std,0.084825
min,0.100000
25%,0.300000
50%,0.300000
75%,0.400000
max,0.600000


### 미세먼지 월별 데이터 불러오기

In [28]:
pm25_file_path = 'C:\\Users\\pwjdg\\바탕 화면\\Green-Car-Charging-Station-Site-Selection\\Data\\미세먼지_PM2.5__월별_도시별_대기오염도.csv'
df_pm25 = pd.read_csv(pm25_file_path, encoding='cp949')
df_pm25

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,25,27,17,19,19,12
1,서울특별시,서울특별시,29,32,19,20,19,15
2,부산광역시,부산광역시,21,21,15,16,15,9
3,대구광역시,대구광역시,24,23,17,18,20,10
4,인천광역시,인천광역시,28,33,19,20,23,19
...,...,...,...,...,...,...,...,...
167,경상남도,합천,18,17,14,15,18,11**
168,경상남도,도평균,19,19,15,15,16,12*
169,제주특별자치도,제주특별자치도,18,24,14,16,14,8
170,제주특별자치도,서귀포,16,23,14,16,12,8


In [29]:
df_pm25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   구분(1)    172 non-null    object
 1   구분(2)    172 non-null    object
 2   2021.02  172 non-null    object
 3   2021.03  172 non-null    object
 4   2021.04  172 non-null    object
 5   2021.05  172 non-null    object
 6   2021.06  172 non-null    object
 7   2021.07  172 non-null    object
dtypes: object(8)
memory usage: 10.9+ KB


In [30]:
pm25_col = df_pm25.iloc[:,2:-1].columns.tolist()

In [31]:
df_pm25.drop(columns=pm25_col, inplace=True)

In [32]:
advanced_replace(df_pm25, '2021.07','-',r'[^0-9.0-9]')
df_pm25['2021.07'] = df_pm25['2021.07'].astype(float)

In [33]:
df_pm25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [34]:
df_pm25.describe()

,2021.07
count,172.000000
mean,11.558140
std,2.888188
min,0.000000
25%,10.000000
50%,11.000000
75%,13.000000
max,19.000000


In [35]:
pm10_file_path = 'C:\\Users\\pwjdg\\바탕 화면\\Green-Car-Charging-Station-Site-Selection\\Data\\미세먼지_PM10__월별_도시별_대기오염도.csv'
df_pm10 = pd.read_csv(pm10_file_path, encoding='cp949')
df_pm10

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,43,65,41,52,32,22
1,서울특별시,서울특별시,48,67,42,61,33,24
2,부산광역시,부산광역시,37,60,37,36,26,18
3,대구광역시,대구광역시,41,59,42,41,35,19
4,인천광역시,인천광역시,46,69,42,59,36,29
...,...,...,...,...,...,...,...,...
167,경상남도,합천,29,44,32,36,27,17**
168,경상남도,도평균,33,53,34,36,27,20
169,제주특별자치도,제주특별자치도,36,76,37,44,27,17
170,제주특별자치도,서귀포,33,68,35,40,23,21


In [36]:
pm10_col = df_pm10.iloc[:,2:-1].columns.tolist()

In [37]:
df_pm10.drop(columns=pm10_col, inplace=True)

In [38]:
advanced_replace(df_pm10, '2021.07', '-', r'[^0-9.0-9]')
df_pm10['2021.07'] = df_pm10['2021.07'].astype(float)

In [39]:
df_pm10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [40]:
df_pm10.describe()

,2021.07
count,172.000000
mean,20.750000
std,4.725352
min,0.000000
25%,18.000000
50%,20.000000
75%,23.000000
max,36.000000


### 부산광역시 공영주차장 정보 불러오기

In [41]:
parkinglot_file_path = "C:\\Users\\pwjdg\\바탕 화면\\Green-Car-Charging-Station-Site-Selection\\Data\\부산광역시 공영주차장 정보 조회.csv"
df_parkinglot = pd.read_csv(parkinglot_file_path,encoding='cp949')
df_parkinglot

,운영형태,관리기관명,주차장명,주차장관리번호,소재지도로명주소,주차장구분,주차장유형,주차구획수,위도,경도
0,구(군)직영,부산광역시 사상구청,감전동주변,2010000013,사상구 감전동 589-2,공영,노외,26,35.154108,128.980071
1,구(군)직영,부산광역시 사상구청,장성주차장,2010000015,사상구 주례2동 92-278,공영,노외,16,0.000000,0.000000
2,민간위탁,부산광역시 사상구청,엄궁초등학교지하주차장,2010000016,사상구 엄궁로191번길 23,공영,노외,93,35.128446,128.973403
3,구(군)직영,부산광역시 사상구청,일신주택주변,2011000003,사상구 주례3동 545-3,공영,노외,10,35.145207,129.000672
4,구(군)직영,부산광역시 사상구청,주례중학교주변,2012000004,사상구 주례2동 82-13,공영,노외,8,35.154384,129.011131
...,...,...,...,...,...,...,...,...,...,...
609,구(군)직영,부산광역시 기장군청,정관읍 상업지역 노상주차장,2016000017,기장군 정관면 732,공영,노상,46,0.000000,0.000000
610,민간운영,부산광역시 기장군청,1구역 2소,2008000002,기장군 기장읍 280-2,공영,노상,12,35.222141,129.206947
611,민간운영,부산광역시 기장군청,1구역 3소,2008000003,기장군 기장읍 444-1,공영,노상,13,35.222141,129.206947
612,민간운영,부산광역시 기장군청,3구역 1소,2008000006,기장군 기장읍 418-12,공영,노상,24,35.222141,129.206947


In [54]:
df_parkinglot['주차구획수'].describe()

count    614.000000
mean      62.245928
std       70.352247
min       -4.000000
25%       18.000000
50%       37.500000
75%       78.750000
max      414.000000
Name: 주차구획수, dtype: float64

In [49]:
parkinglot_group = df_parkinglot.groupby('관리기관명')['주차구획수'].sum()
parkinglot_group

관리기관명
부산광역시 강서구청      847
부산광역시 금정구청     4494
부산광역시 기장군청     1515
부산광역시 남구청      1217
부산광역시 동구청      2496
부산광역시 동래구청     1709
부산광역시 부산진구청    2496
부산광역시 북구청      2180
부산광역시 사상구청     5671
부산광역시 사하구청     3961
부산광역시 서구청      2500
부산광역시 수영구청      871
부산광역시 연제구청      655
부산광역시 영도구청     1691
부산광역시 중구청      1481
부산광역시 해운대구청    4353
Name: 주차구획수, dtype: int64

## 상위 요소 - 사회적 요인

### 행정구역 인구 데이터 불러오기

In [42]:
people_file_path = 'C:\\Users\\pwjdg\\바탕 화면\\Green-Car-Charging-Station-Site-Selection\\Data\\행정구역_시군구_별__성별_인구수.csv'
df_people = pd.read_csv(people_file_path, encoding='cp949', header=1)
df_people

,행정구역(시군구)별,총인구수 (명)
0,전국,51592660
1,서울특별시,9500480
2,종로구,144005
3,중구,122087
4,용산구,222009
...,...,...
287,거창군,60756
288,합천군,42613
289,제주특별자치도,677413
290,제주시,493076


In [43]:
df_people['총인구수 (명)'] = df_people['총인구수 (명)'].astype(float)

In [44]:
df_people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   행정구역(시군구)별  292 non-null    object 
 1   총인구수 (명)    292 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.7+ KB


In [45]:
# 0번 행(전국 수치) 삭제
df_people.drop([0],inplace=True)

In [46]:
df_people

,행정구역(시군구)별,총인구수 (명)
1,서울특별시,9500480.0
2,종로구,144005.0
3,중구,122087.0
4,용산구,222009.0
5,성동구,284573.0
...,...,...
287,거창군,60756.0
288,합천군,42613.0
289,제주특별자치도,677413.0
290,제주시,493076.0


In [47]:
# 부산에 대한 통계 요약.
busan_people_describe  = df_people.iloc[27:43, :].describe()
busan_people_describe

,총인구수 (명)
count,16.000000
mean,208763.375000
std,98697.592227
min,40585.000000
25%,135080.250000
50%,206521.000000
75%,275695.000000
max,393130.000000


### 전기차 충전소 데이터 불러오기

In [56]:
evcs_file_path = "C:\\Users\\pwjdg\\바탕 화면\\Green-Car-Charging-Station-Site-Selection\\Data\\전기차 충전소 리스트.csv"
df_evcs = pd.read_csv(evcs_file_path, encoding='cp949')
df_evcs

,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소
0,환경부(한국자동차환경협회),신북보건지소공영주차장,21,DC콤보,전라남도,영암군,전라남도 영암군 신북면 간은정로 21-7
1,환경부(한국자동차환경협회),신북보건지소공영주차장,22,DC콤보,전라남도,영암군,전라남도 영암군 신북면 간은정로 21-7
2,환경부(한국자동차환경협회),한국생산기술연구원 대경본부바이오메디칼생산기술센터,1,DC콤보,경상북도,영천시,경상북도 영천시 양호길 59
3,환경부(한국자동차환경협회),한국생산기술연구원 대경본부바이오메디칼생산기술센터,2,DC콤보,경상북도,영천시,경상북도 영천시 양호길 59
4,환경부(한국자동차환경협회),(구)거진수산센터 주차장,11,DC콤보,강원도,고성군,강원도 고성군 거진읍 거진리 22-93
...,...,...,...,...,...,...,...
119204,한화솔루션,한국과학기술회관,3,AC완속,서울특별시,강남구,서울특별시 강남구 테헤란로7길 22한국과학기술회관 지하 2~4층(635-2 과학기술...
119205,한화솔루션,한국과학기술회관,4,AC완속,서울특별시,강남구,서울특별시 강남구 테헤란로7길 22한국과학기술회관 지하 2~4층(635-2 과학기술...
119206,한화솔루션,한국과학기술회관,5,AC완속,서울특별시,강남구,서울특별시 강남구 테헤란로7길 22한국과학기술회관 지하 2~4층(635-2 과학기술...
119207,한화솔루션,한국과학기술회관,6,AC완속,서울특별시,강남구,서울특별시 강남구 테헤란로7길 22한국과학기술회관 지하 2~4층(635-2 과학기술...


### 수소차 충전소 데이터 불러오기

In [59]:
hvcs_file_path = 'C:\\Users\\pwjdg\\바탕 화면\\Green-Car-Charging-Station-Site-Selection\\Data\\수소차 충전소.csv'
df_hvcs = pd.read_csv(hvcs_file_path, encoding='cp949')
df_hvcs

,충전소,충전기,주소,위도,경도
0,상암수소충전소,1,서울특별시 마포구 하늘공원로 108,37.569685,126.880073
1,마곡에코수소충전소,1,서울특별시 강서구 양천로 225,37.573016,126.829208
2,오곡수소충전소,1,서울특별시 강서구 벌말로 271,37.555845,126.768181
3,H국회수소충전소,1,서울특별시 영등포구 의사당대로 1,37.532912,126.916404
4,양재수소충전소,1,서울특별시 서초구 바우뫼로12길 73,37.468378,127.034374
...,...,...,...,...,...
105,하이넷 통영용남수소충전소,1,경상남도 통영시 용남면 남해안대로 38,34.891246,128.471819
106,김해수소충전소,2,경상남도 김해시 김해대로2596번길 53,0.000000,0.000000
107,하이넷 거제장평수소충전소,1,경상남도 거제시 거제대로 5050,34.894764,128.587447
108,양산시증산수소충전소,1,경상남도 양산시 물금읍 메기로 114,35.303404,129.010257
